In [1]:
import mlflow
# Step 2: Set up the MLflow tracking server
mlflow.set_tracking_uri("http://ec2-98-93-89-136.compute-1.amazonaws.com:5000/")

In [2]:
# Set or create an experiment
mlflow.set_experiment("04 - Imbalance Data Handling Experiments")

2025/11/22 15:10:24 INFO mlflow.tracking.fluent: Experiment with name '04 - Imbalance Data Handling Experiments' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://sau-s3-mlflow-bucket/714158397471718545', creation_time=1763842224323, experiment_id='714158397471718545', last_update_time=1763842224323, lifecycle_stage='active', name='04 - Imbalance Data Handling Experiments', tags={}>

In [4]:
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import mlflow
import mlflow.sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

In [5]:
df = pd.read_csv(
    '../../data/processed/reddit_clean_final.csv',
    keep_default_na=False,
    na_filter=False
)
df.shape

(36607, 2)

In [8]:
# Step 1: Function to run the experiment
def run_imbalanced_experiment(imbalance_method):
    ngram_range = (1, 2)  # Bigram setting
    max_features = 10000  # Set max_features to 1000 for TF-IDF

    # Step 4: Train-test split before vectorization and resampling
    X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], test_size=0.2, random_state=42, stratify=df['category'])

    # Step 2: Vectorization using TF-IDF, fit on training data only
    vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
    X_train_vec = vectorizer.fit_transform(X_train)  # Fit on training data
    X_test_vec = vectorizer.transform(X_test)  # Transform test data

    # Step 3: Handle class imbalance based on the selected method (only applied to the training set)
    if imbalance_method == 'class_weights':
        # Use class_weight in Random Forest
        class_weight = 'balanced'
    else:
        class_weight = None  # Do not apply class_weight if using resampling

        # Resampling Techniques (only apply to the training set)
        if imbalance_method == 'oversampling':
            smote = SMOTE(random_state=42)
            X_train_vec, y_train = smote.fit_resample(X_train_vec, y_train)
        elif imbalance_method == 'adasyn':
            adasyn = ADASYN(random_state=42)
            X_train_vec, y_train = adasyn.fit_resample(X_train_vec, y_train)
        elif imbalance_method == 'undersampling':
            rus = RandomUnderSampler(random_state=42)
            X_train_vec, y_train = rus.fit_resample(X_train_vec, y_train)
        elif imbalance_method == 'smote_enn':
            smote_enn = SMOTEENN(random_state=42)
            X_train_vec, y_train = smote_enn.fit_resample(X_train_vec, y_train)

    # Step 5: Define and train a Random Forest model
    with mlflow.start_run() as run:
        # Set tags for the experiment and run
        mlflow.set_tag("mlflow.runName", f"Imbalance_{imbalance_method}_RandomForest_TFIDF_Bigrams")
        mlflow.set_tag("experiment_type", "imbalance_handling")
        mlflow.set_tag("model_type", "RandomForestClassifier")

        # Add a description
        mlflow.set_tag("description", f"RandomForest with TF-IDF Bigrams, imbalance handling method={imbalance_method}")

        # Log vectorizer parameters
        mlflow.log_param("vectorizer_type", "TF-IDF")
        mlflow.log_param("ngram_range", ngram_range)
        mlflow.log_param("vectorizer_max_features", max_features)

        # Log Random Forest parameters
        n_estimators = 200
        max_depth = 15

        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)
        mlflow.log_param("imbalance_method", imbalance_method)

        # Initialize and train the model
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42, class_weight=class_weight)
        model.fit(X_train_vec, y_train)

        # Step 6: Make predictions and log metrics
        y_pred = model.predict(X_test_vec)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log confusion matrix
        conf_matrix = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix: TF-IDF Bigrams, Imbalance={imbalance_method}")
        confusion_matrix_filename = f"04_confusion_matrix_{imbalance_method}.png"
        save_path = f"../../images/evaluation/{confusion_matrix_filename}"
        plt.savefig(save_path)
        mlflow.log_artifact(save_path)
        plt.close()

        # Log the model
        mlflow.sklearn.log_model(model, f"random_forest_model_tfidf_bigrams_imbalance_{imbalance_method}")

# Step 7: Run experiments for different imbalance methods
imbalance_methods = ['class_weights', 'oversampling', 'adasyn', 'undersampling', 'smote_enn']

for method in imbalance_methods:
    run_imbalanced_experiment(method)


d:\SAURABH\spring 2024\MLOps Project\CreatorInsight AI Platform\.venv\lib\site-packages\boto3\compat.py:84: PythonDeprecationWarning: Boto3 will no longer support Python 3.9 starting April 29, 2026. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.10 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)
2025/11/22 15:24:48 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/22 15:25:10 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Imbalance_class_weights_RandomForest_TFIDF_Bigrams at: http://ec2-98-93-89-136.compute-1.amazonaws.com:5000/#/experiments/714158397471718545/runs/9d9b577bef8c4115ba0279c9c8c2b90b
🧪 View experiment at: http://ec2-98-93-89-136.compute-1.amazonaws.com:5000/#/experiments/714158397471718545


d:\SAURABH\spring 2024\MLOps Project\CreatorInsight AI Platform\.venv\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
d:\SAURABH\spring 2024\MLOps Project\CreatorInsight AI Platform\.venv\lib\site-packages\boto3\compat.py:84: PythonDeprecationWarning: Boto3 will no longer support Python 3.9 starting April 29, 2026. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.10 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)
2025/11/22 15:32:41 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/22 15:33:04 WARNING mlflow.models.model: Mod

🏃 View run Imbalance_oversampling_RandomForest_TFIDF_Bigrams at: http://ec2-98-93-89-136.compute-1.amazonaws.com:5000/#/experiments/714158397471718545/runs/1bdc177b31424f8189c393cd7a4a2019
🧪 View experiment at: http://ec2-98-93-89-136.compute-1.amazonaws.com:5000/#/experiments/714158397471718545


d:\SAURABH\spring 2024\MLOps Project\CreatorInsight AI Platform\.venv\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
d:\SAURABH\spring 2024\MLOps Project\CreatorInsight AI Platform\.venv\lib\site-packages\boto3\compat.py:84: PythonDeprecationWarning: Boto3 will no longer support Python 3.9 starting April 29, 2026. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.10 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)
2025/11/22 15:35:36 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/22 15:35:53 WARNING mlflow.models.model: Mod

🏃 View run Imbalance_adasyn_RandomForest_TFIDF_Bigrams at: http://ec2-98-93-89-136.compute-1.amazonaws.com:5000/#/experiments/714158397471718545/runs/ca48ca4bb00540fdb2e1e2ad6da2afed
🧪 View experiment at: http://ec2-98-93-89-136.compute-1.amazonaws.com:5000/#/experiments/714158397471718545


d:\SAURABH\spring 2024\MLOps Project\CreatorInsight AI Platform\.venv\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
d:\SAURABH\spring 2024\MLOps Project\CreatorInsight AI Platform\.venv\lib\site-packages\sklearn\base.py:484: FutureWarning: `BaseEstimator._check_n_features` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_n_features` instead.
  warnings.warn(
d:\SAURABH\spring 2024\MLOps Project\CreatorInsight AI Platform\.venv\lib\site-packages\sklearn\base.py:493: FutureWarning: `BaseEstimator._check_feature_names` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_feature_names` instead.
  warnings.warn(
2025/11/22 15:36:32 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/22 15:36:45 WARNING mlflow.models.model: Model logged without a signatur

🏃 View run Imbalance_undersampling_RandomForest_TFIDF_Bigrams at: http://ec2-98-93-89-136.compute-1.amazonaws.com:5000/#/experiments/714158397471718545/runs/bba88567647041f18c447efe785f3595
🧪 View experiment at: http://ec2-98-93-89-136.compute-1.amazonaws.com:5000/#/experiments/714158397471718545


d:\SAURABH\spring 2024\MLOps Project\CreatorInsight AI Platform\.venv\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
d:\SAURABH\spring 2024\MLOps Project\CreatorInsight AI Platform\.venv\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
d:\SAURABH\spring 2024\MLOps Project\CreatorInsight AI Platform\.venv\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part

🏃 View run Imbalance_smote_enn_RandomForest_TFIDF_Bigrams at: http://ec2-98-93-89-136.compute-1.amazonaws.com:5000/#/experiments/714158397471718545/runs/a450d85062944eb9bd9d5d7cb1c4625d
🧪 View experiment at: http://ec2-98-93-89-136.compute-1.amazonaws.com:5000/#/experiments/714158397471718545
